In [ ]:
from pipeline.staging.extract import extract_database, extract_list_table, extract_sheet
from pipeline.staging.load import load_staging
from utils.db_conn import db_connection
from utils.log import etl_log
import pandas as pd

In [ ]:
tables_to_extract = ['acquisition',
                    'company',
                    'funding_rounds',
                    'funds',
                    'investments',
                    'ipos']

excel_to_extract = ['people',
                    'relationships']

extract_data = [
                'acquisition',
                'company',
                'funding_rounds',
                'funds',
                'investments',
                'ipos',
                'people',
                'relationships'
                ]

table_id = [
            'acquisition_id',
            'office_id',
            'funding_round_id',
            'fund_id',
            'investment_id',
            'ipo_id',
            'people_id',
            'relationship_id'
]

In [ ]:
for index, table_name in enumerate(tables_to_extract):
    table_extract = extract_database(db_name='startup_investments', table_name = table_name)
            

In [ ]:
for index, file_name in enumerate(excel_to_extract):
    excel_extract = extract_sheet(file_name)




In [ ]:
load_staging(table_name='acquisition', idx_name='acquisition_id', source='startup investment')

In [ ]:
src_engine,_,_ = db_connection()

In [ ]:
try:
    with src_engine.connect() as connection:
        print("Database connection successful!")

except Exception as e:
    print(f"Database connection failed: {e}")


In [ ]:
log_msg = {
        "step" : "staging",
        "process":"extraction",
        "status": "success",
        "source": "database",
        "table_name": "test" # Current timestamp
    }

In [ ]:
etl_log(log_msg)

In [ ]:
load_staging(table_name='company', idx_name='office_id', source='startup investment')

In [ ]:
for index, table_name in enumerate(tables_to_extract):
    load_staging(table_name= table_name, idx_name='acquisition_id', source='startup investment')
    table_extract = extract_database(db_name='startup_investments', table_name = table_name)

In [ ]:
for index, table_name in enumerate(extract_data):
    load_staging(table_name= table_name, idx_name=table_id[index], source='startup investment')

In [ ]:
for index, table_name in enumerate(extract_data):
    print(table_name)
    print(table_id[index])
    print("======")

In [1]:
from pipeline.dwh.extract import extract_database

In [2]:
extract_data = [
                'acquisition',
                'company',
                'funding_rounds',
                'funds',
                'investments',
                'ipos',
                'people',
                'relationships'
                ]

In [ ]:
for index, table_name in enumerate(extract_data):
    table_extract = extract_database(table_name = table_name)

TypeError: extract_database() got an unexpected keyword argument 'db_name'

In [ ]:
table_extract = extract_database(table_name = 'company')

In [1]:
import pandas as pd
from utils.db_conn import db_connection

from utils.log import etl_log, read_etl_log


In [2]:
_,_ ,stg_engine= db_connection()

In [6]:
# Get date from previous process
filter_log = {"step_name": "warehouse",
            "table_name": "company",
            "status": "success",
            "process": "load"}
etl_date = read_etl_log(filter_log)
if(etl_date['max'][0] == None):
    etl_date = '1111-01-01'
else:
    etl_date = etl_date[max][0]

query = f"SELECT * FROM company WHERE created_at  > %s::timestamp"
df = pd.read_sql(sql=query, con=stg_engine, params=(etl_date,))
df.to_csv(f"../data-pipeline/pipeline/temp/company.csv", index=False)

In [ ]:
from utils.db_conn import dwh_connection
dwh_engine = dwh_connection()

In [ ]:


try:
    with dwh_engine.connect() as connection:
        print("Database connection successful!")

except Exception as e:
    print(f"Database connection failed: {e}")

In [ ]:
from pipeline.dwh.transform import transform_commpany
from pipeline.dwh.load import load_warehouse



In [ ]:
company_transform = transform_commpany(table_name='company')


In [ ]:
print(company_transform)

In [ ]:
load_warehouse(data=company_transform, table_name='dim_company', idx_name='office_nk', table_process='company', source='staging')